In [16]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import json
import time
import numpy as np
from IPython.display import display
import ipywidgets as widgets
import holidays
import xgboost as xgb

In [17]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/A Round Entertainment/"

df = pd.read_csv('combined_df.csv')

# Filter the dataset without dropping NaN values (for training set)
filtered_data = df[
    (df['Year'] >= 2020) &
    (~(df['Headliner'].str.contains('"', na=False))) &
    (df['Genre'] != 'Family Entertainment') &
    (df['Ticket Price Min USD'] > 0) &
    (df['Ticket Price Min USD'] < df['Ticket Price Max USD'])
]

# Filter the dataset with NaN rows dropped (for testing set)
filtered_data_no_na = filtered_data.dropna()

# Split 30% of the cleaned dataset (no NaNs) into the testing set
_, test_data = train_test_split(filtered_data_no_na, test_size=0.3, random_state=42)

# Ensure the training set is mutually exclusive by removing test rows from the original filtered dataset
train_data = filtered_data.loc[~filtered_data.index.isin(test_data.index)]

# Check the results
print(f"Total filtered data size: {len(filtered_data)}")
print(f"Training set size (including NaNs): {len(train_data)}")
print(f"Testing set size (no NaNs): {len(test_data)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/A Round Entertainment


<ipython-input-17-81346a7b7cd1>:5: DtypeWarning: Columns (2,3,6,7,8,9,10,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined_df.csv')


Total filtered data size: 79691
Training set size (including NaNs): 78682
Testing set size (no NaNs): 1009


In [26]:
feature_columns = ['sp followers', 'sp popularity', 'yt View Count', 'yt Subscriber Count', 'Total population',
                   'monthly_listeners', 'Number of Shows', 'Avg. Event Capacity', 'Ticket Price Min USD', 'Ticket Price Max USD']

X_train = train_data[feature_columns]
y_train = train_data['Avg. Gross USD']
X_test = test_data[feature_columns]
y_test = test_data['Avg. Gross USD']

# **XGBoost**

In [ ]:
# MAE
mae_scorer = make_scorer(mean_absolute_error)
# MSE
mse_scorer = make_scorer(mean_squared_error)
# R²
r2_scorer = make_scorer(r2_score)


# XGBoost (tune hyperparameter)
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    max_depth=6,
    learning_rate=0.1,
    n_estimators=1000,
    random_state=42
)

#Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

r2_scores = []
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Train the model
    xgb_model.fit(X_train_fold, y_train_fold)

    # Make predictions
    y_val_pred = xgb_model.predict(X_val_fold)

    # Calculate R²
    r2 = r2_score(y_val_fold, y_val_pred)
    r2_scores.append(r2)

# Calculate the average R² across all folds
avg_r2 = np.mean(r2_scores)
print(f"Cross-validation R² scores: {r2_scores}")
print(f"Average Cross-validation R²: {avg_r2:.4f}")

# Train the model on the full training data
xgb_model.fit(X_train, y_train)

#Make predictions on the testing set
y_test_pred = xgb_model.predict(X_test)

#Calculate RMSE and R² on the testing set
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test R²: {test_r2:.4f}")

Cross-validation R² scores: [0.8928768060911378, 0.9141210899022755, 0.9253521917100389, 0.9273271225984516, 0.9141405940658137]
Average Cross-validation R²: 0.9148
